# Nerea Del Olmo Sanz

In [1]:
%load_ext sql
%sql sqlite:///examen1_baloncesto_sql.db

/usr/local/lib/python3.5/dist-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/usr/local/lib/python3.5/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


'Connected: None@examen1_baloncesto_sql.db'

En primer lugar vamos a realizar sólo una pequeña parte del diagrama E/R total, la cual se puede ver en la siguiente imagen. (Son los puntos 1, 2, 6 y 7 del enunciado).
**NOTA**: Cancha debe estar subrayado porque es la clave primaria (este error ocurre en algunos más diagramas).
![Diagrama E/R](Diagrama1.jpeg)

In [2]:
%sql PRAGMA foreign_keys = ON

Done.


[]

Creamos las tablas de Comunidades, Ciudades y Equipos. Puesto que todas son entidades débiles de la superior, no necesitamos tablas para las relaciones entre dichas entidades, añadiendo la restricción NOT NULL.

In [4]:
%%sql
CREATE TABLE Comunidades(
    comunidad VARCHAR(30),
    PRIMARY KEY (comunidad))

Done.


[]

In [5]:
%%sql
INSERT INTO Comunidades(comunidad) VALUES ('madrid');
INSERT INTO Comunidades(comunidad) VALUES ('galicia');
INSERT INTO Comunidades(comunidad) VALUES ('cantabria');

1 rows affected.
1 rows affected.
1 rows affected.


[]

In [3]:
%sql SELECT * FROM Comunidades

Done.


comunidad
madrid
galicia
cantabria


In [7]:
%%sql
CREATE TABLE Ciudades(
    ciudad VARCHAR(30),
    comunidad VARCHAR(30) NOT NULL,
    PRIMARY KEY (ciudad, comunidad),
    FOREIGN KEY (comunidad) REFERENCES Comunidades(comunidad) ON DELETE CASCADE)

Done.


[]

In [8]:
%%sql
INSERT INTO Ciudades(ciudad, comunidad) VALUES ('alcobendas','madrid');
INSERT INTO Ciudades(ciudad, comunidad) VALUES ('lugo','galicia');
INSERT INTO Ciudades(ciudad, comunidad) VALUES ('pontevedra','galicia');
INSERT INTO Ciudades(ciudad, comunidad) VALUES ('alcobendas','cantabria');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [4]:
%sql SELECT * FROM Ciudades

Done.


ciudad,comunidad
alcobendas,madrid
lugo,galicia
pontevedra,galicia
alcobendas,cantabria


In [16]:
%sql DROP TABLE Equipos

Done.


[]

In [97]:
%%sql
CREATE TABLE Equipos(
    equipo VARCHAR(30),
    ciudad VARCHAR(30) NOT NULL,
    comunidad VARCHAR(30) NOT NULL,
    PRIMARY KEY (equipo, ciudad, comunidad),
    FOREIGN KEY (ciudad,comunidad) REFERENCES Ciudades(ciudad,comunidad) ON DELETE CASCADE)


Done.


[]

In [98]:
%%sql
INSERT INTO Equipos(equipo, ciudad, comunidad)
VALUES ('atletico', 'lugo', 'galicia');
INSERT INTO Equipos(equipo, ciudad, comunidad)
VALUES ('atletico', 'alcobendas', 'madrid');
INSERT INTO Equipos(equipo, ciudad, comunidad)
VALUES ('real', 'alcobendas', 'madrid');
INSERT INTO Equipos(equipo, ciudad, comunidad)
VALUES ('deportivo', 'pontevedra', 'galicia');
INSERT INTO Equipos(equipo, ciudad, comunidad)
VALUES ('betis', 'alcobendas', 'cantabria');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [124]:
%sql SELECT * FROM Equipos

Done.


equipo,ciudad,comunidad
atletico,lugo,galicia
atletico,alcobendas,madrid
real,alcobendas,madrid
deportivo,pontevedra,galicia
betis,alcobendas,cantabria


Ahora, suponemos que un nombre de cancha no puede repetirse nunca, ni en distintas ciudades. De este modo, podemos tomarlo como clave primaria de la entidad.

In [15]:
%%sql
CREATE TABLE Canchas(
    cancha VARCHAR(30),
    PRIMARY KEY (cancha));

Done.


[]

In [16]:
%%sql
INSERT INTO Canchas(cancha) VALUES ('cancha1');
INSERT INTO Canchas(cancha) VALUES ('cancha2');
INSERT INTO Canchas(cancha) VALUES ('cancha3');

1 rows affected.
1 rows affected.
1 rows affected.


[]

In [17]:
%sql SELECT * FROM Canchas

Done.


cancha
cancha1
cancha2
cancha3


Puesto que las canchas y sus direcciones son una relación 1-to-1, podemos ahorrarnos la tabla "ubicada_en".

In [26]:
%sql DROP TABLE Direcciones_de_canchas

Done.


[]

In [27]:
%%sql
CREATE TABLE Direcciones_de_canchas(
    cancha VARCHAR(30) NOT NULL,
    calle VARCHAR(30) NOT NULL,
    numero INT NOT NULL,
    ciudad VARCHAR(30) NOT NULL,
    comunidad VARCHAR(30) NOT NULL,
    PRIMARY KEY (cancha, calle, numero, ciudad, comunidad),
    FOREIGN KEY (cancha) REFERENCES Canchas(cancha) ON DELETE CASCADE,
    FOREIGN KEY (ciudad,comunidad) REFERENCES Ciudades(ciudad,comunidad) ON DELETE CASCADE)

Done.


[]

In [28]:
%%sql
INSERT INTO Direcciones_de_canchas(cancha, calle, numero, ciudad, comunidad)
VALUES ('cancha1', 'cervantes', 22 , 'lugo', 'galicia');
INSERT INTO Direcciones_de_canchas(cancha, calle, numero, ciudad, comunidad)
VALUES ('cancha2', 'felicidad', 156 , 'alcobendas', 'cantabria');
INSERT INTO Direcciones_de_canchas(cancha, calle, numero, ciudad, comunidad)
VALUES ('cancha3', 'naranjos', 35 , 'pontevedra', 'galicia');

1 rows affected.
1 rows affected.
1 rows affected.


[]

In [29]:
%sql SELECT * FROM Direcciones_de_canchas

Done.


cancha,calle,numero,ciudad,comunidad
cancha1,cervantes,22,lugo,galicia
cancha2,felicidad,156,alcobendas,cantabria
cancha3,naranjos,35,pontevedra,galicia


In [34]:
%sql DROP TABLE Entrenan_en

Done.


[]

In [135]:
%%sql
CREATE TABLE Entrenan_en(
    equipo VARCHAR(30),
    ciudad VARCHAR(30),
    comunidad VARCHAR(30),
    cancha VARCHAR(30),    
    PRIMARY KEY (equipo, ciudad, comunidad, cancha),
    FOREIGN KEY (equipo, ciudad, comunidad) REFERENCES Equipos(equipo,ciudad,comunidad) ON DELETE CASCADE)

(sqlite3.OperationalError) table Entrenan_en already exists [SQL: 'CREATE TABLE Entrenan_en(\n    equipo VARCHAR(30),\n    ciudad VARCHAR(30),\n    comunidad VARCHAR(30),\n    cancha VARCHAR(30),    \n    PRIMARY KEY (equipo, ciudad, comunidad, cancha),\n    FOREIGN KEY (equipo, ciudad, comunidad) REFERENCES Equipos(equipo,ciudad,comunidad) ON DELETE CASCADE)']


In [136]:
%%sql
INSERT INTO Entrenan_en(equipo, ciudad, comunidad, cancha)
VALUES ('atletico', 'lugo', 'galicia', 'cancha1');
INSERT INTO Entrenan_en(equipo, ciudad, comunidad, cancha)
VALUES ('atletico', 'alcobendas', 'madrid','cancha2');
INSERT INTO Entrenan_en(equipo, ciudad, comunidad, cancha)
VALUES ('real', 'alcobendas', 'madrid','cancha2');
INSERT INTO Entrenan_en(equipo, ciudad, comunidad, cancha)
VALUES ('deportivo', 'pontevedra', 'galicia', 'cancha1');

INSERT INTO Entrenan_en(equipo, ciudad, comunidad, cancha)
VALUES ('betis', 'alcobendas', 'cantabria','cancha3');

INSERT INTO Entrenan_en(equipo, ciudad, comunidad, cancha)
VALUES ('betis', 'alcobendas', 'cantabria', 'cancha2');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [137]:
%sql SELECT * FROM Entrenan_en

Done.


equipo,ciudad,comunidad,cancha
atletico,lugo,galicia,cancha1
atletico,alcobendas,madrid,cancha2
real,alcobendas,madrid,cancha2
deportivo,pontevedra,galicia,cancha1
betis,alcobendas,cantabria,cancha3
betis,alcobendas,cantabria,cancha2


Ahora, añadimos los puntos 3 y 4 del enunciado.
![Diagrama E/R](Diagrama2.jpeg)

In [38]:
%%sql
CREATE TABLE Empresas(
    empresa VARCHAR(30),
    cif VARCHAR(30),
    PRIMARY KEY (cif));

(sqlite3.OperationalError) table Empresas already exists [SQL: 'CREATE TABLE Empresas(\n    empresa VARCHAR(30),\n    cif VARCHAR(30),\n    PRIMARY KEY (cif));']


In [39]:
%%sql
INSERT INTO Empresas(empresa, cif)
VALUES ('indra','A11111111');
INSERT INTO Empresas(empresa, cif)
VALUES ('telefonica','B22222222');
INSERT INTO Empresas(empresa, cif)
VALUES ('inditex','B33333333');

1 rows affected.
1 rows affected.
1 rows affected.


[]

In [40]:
%sql SELECT * FROM Empresas;

Done.


empresa,cif
indra,A11111111
telefonica,B22222222
inditex,B33333333


In [23]:
%%sql
CREATE TABLE Temporadas(
    f_inicio DATETIME,
    f_fin DATETIME,
    cif VARCHAR(30) NOT NULL,
    PRIMARY KEY (f_inicio),
    FOREIGN KEY (cif) REFERENCES Empresas(cif) ON DELETE CASCADE);

Done.


[]

In [41]:
%%sql
INSERT INTO Temporadas(f_inicio, f_fin, cif)
VALUES ('2007-01-1 00:00:00', '2007-12-31 00:00:00','A11111111');
INSERT INTO Temporadas(f_inicio, f_fin, cif)
VALUES ('2008-01-1 00:00:00', '2008-12-31 00:00:00','B22222222');
INSERT INTO Temporadas(f_inicio, f_fin, cif)
VALUES ('2009-01-1 00:00:00', '2009-12-31 00:00:00','B33333333');

1 rows affected.
1 rows affected.
1 rows affected.


[]

In [42]:
%sql SELECT * FROM Temporadas;

Done.


f_inicio,f_fin,cif
2007-01-1 00:00:00,2007-12-31 00:00:00,A11111111
2008-01-1 00:00:00,2008-12-31 00:00:00,B22222222
2009-01-1 00:00:00,2009-12-31 00:00:00,B33333333


Ahora, añadimos el punto 8 del enunciado.
![Diagrama E/R](Diagrama3.jpeg)

In [45]:
%sql DROP TABLE Financian

Done.


[]

In [46]:
%%sql
CREATE TABLE Financian(
    equipo VARCHAR(30),
    ciudad VARCHAR(30),
    comunidad VARCHAR(30),
    cif VARCHAR(30),
    PRIMARY KEY (equipo, ciudad, comunidad, cif),
    FOREIGN KEY (equipo, ciudad, comunidad) REFERENCES Equipos(equipo, ciudad, comunidad) ON DELETE CASCADE,
    FOREIGN KEY (ciudad, comunidad) REFERENCES Ciudades(ciudad, comunidad) ON DELETE CASCADE,
    FOREIGN KEY (cif) REFERENCES Empresas(cif) ON DELETE CASCADE);

Done.


[]

In [47]:
%%sql
INSERT INTO Financian(equipo, ciudad, comunidad, cif)
VALUES ('betis', 'alcobendas', 'cantabria','A11111111');
INSERT INTO Financian(equipo, ciudad, comunidad, cif)
VALUES ('atletico', 'lugo', 'galicia','B22222222');
INSERT INTO Financian(equipo, ciudad, comunidad, cif)
VALUES ('atletico', 'alcobendas', 'madrid','B33333333');
INSERT INTO Financian(equipo, ciudad, comunidad, cif)
VALUES ('deportivo', 'pontevedra', 'galicia','B33333333');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [49]:
%sql SELECT * FROM Financian

Done.


equipo,ciudad,comunidad,cif
betis,alcobendas,cantabria,A11111111
atletico,lugo,galicia,B22222222
atletico,alcobendas,madrid,B33333333
deportivo,pontevedra,galicia,B33333333


Continuamos añadiendo el punto 5 del enunciado. ![Diagrama E/R](Diagrama4.jpeg)

In [27]:
%%sql
CREATE TABLE Personas (
    nif VARCHAR(30),
    nombre VARCHAR(30),
    apellidos VARCHAR(30),
    f_nacimiento DATETIME,
    PRIMARY KEY (nif))

Done.


[]

In [50]:
%%sql
INSERT INTO Personas(nif, nombre, apellidos, f_nacimiento)
VALUES ('11111111A', 'juan', 'perez martinez', '1980-12-26 00:00:00');
INSERT INTO Personas(nif, nombre, apellidos, f_nacimiento)
VALUES ('22222222B', 'juan', 'perez martinez', '1975-02-17 00:00:00');
INSERT INTO Personas(nif, nombre, apellidos, f_nacimiento)
VALUES ('33333333C', 'pepe', 'perez martinez', '1984-10-15 00:00:00');
INSERT INTO Personas(nif, nombre, apellidos, f_nacimiento)
VALUES ('44444444D', 'paco', 'rodriguez', '1980-04-16 00:00:00');
INSERT INTO Personas(nif, nombre, apellidos, f_nacimiento)
VALUES ('55555555E', 'elena', 'garcia sanz', '1996-03-28 00:00:00');
INSERT INTO Personas(nif, nombre, apellidos, f_nacimiento)
VALUES ('66666666F', 'rosa', 'sanz hernandez', '1981-11-06 00:00:00');
INSERT INTO Personas(nif, nombre, apellidos, f_nacimiento)
VALUES ('77777777G', 'jose', 'sanchez', '1981-12-24 00:00:00');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [51]:
%sql SELECT * FROM Personas;

Done.


nif,nombre,apellidos,f_nacimiento
11111111A,juan,perez martinez,1980-12-26 00:00:00
22222222B,juan,perez martinez,1975-02-17 00:00:00
33333333C,pepe,perez martinez,1984-10-15 00:00:00
44444444D,paco,rodriguez,1980-04-16 00:00:00
55555555E,elena,garcia sanz,1996-03-28 00:00:00
66666666F,rosa,sanz hernandez,1981-11-06 00:00:00
77777777G,jose,sanchez,1981-12-24 00:00:00


In [67]:
%sql DROP TABLE Jugadores

Done.


[]

In [69]:
%%sql
CREATE TABLE Jugadores (
    jugador VARCHAR(30),
    PRIMARY KEY (jugador),
    FOREIGN KEY (jugador) REFERENCES Personas(nif) ON DELETE CASCADE)

Done.


[]

Modificamos ahora la tabla de Jugadores, añadiéndoles el atributo "estatura", según el punto 21

In [70]:
%sql ALTER TABLE Jugadores ADD COLUMN estatura_cm INT

Done.


[]

In [73]:
%%sql
INSERT INTO Jugadores(jugador, estatura_cm)
VALUES ('11111111A',190);
INSERT INTO Jugadores(jugador, estatura_cm)
VALUES ('22222222B', 192);
INSERT INTO Jugadores(jugador, estatura_cm)
VALUES ('33333333C', 205);
INSERT INTO Jugadores(jugador, estatura_cm)
VALUES ('44444444D', 203);
INSERT INTO Jugadores(jugador, estatura_cm)
VALUES ('55555555E', 204);

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [74]:
%sql SELECT * FROM Jugadores;

Done.


jugador,estatura_cm
11111111A,190
22222222B,192
33333333C,205
44444444D,203
55555555E,204


In [30]:
%%sql
CREATE TABLE Entrenadores (
    entrenador VARCHAR(30),
    PRIMARY KEY (entrenador),
    FOREIGN KEY (entrenador) REFERENCES Personas(nif) ON DELETE CASCADE)

Done.


[]

In [75]:
%%sql
INSERT INTO Entrenadores(entrenador)
VALUES ('66666666F');
INSERT INTO Entrenadores(entrenador)
VALUES ('77777777G');

1 rows affected.
1 rows affected.


[]

In [76]:
%sql SELECT * FROM Entrenadores;

Done.


entrenador
66666666F
77777777G


Añadimos ahora los puntos 9, 10, 11 y 12. ![Diagrama E/R](Diagrama5.jpeg)

In [128]:
%sql DROP TABLE Entrenan

Done.


[]

In [129]:
%%sql
CREATE TABLE Entrenan(
    f_inicio DATETIME,
    f_fin DATETIME,
    entrenador VARCHAR(30),
    equipo VARCHAR(30),
    ciudad VARCHAR(30),
    comunidad VARCHAR(30),
    cancha VARCHAR(30),
    f_temporada DATETIME,
    PRIMARY KEY (f_inicio, entrenador, equipo, ciudad, comunidad, cancha, f_temporada),
    FOREIGN KEY (entrenador) REFERENCES Entrenadores(entrenador) ON DELETE CASCADE,
    FOREIGN KEY (equipo,ciudad,comunidad) REFERENCES Equipos(equipo,ciudad,comunidad) ON DELETE CASCADE,
    
    FOREIGN KEY (cancha) REFERENCES Canchas(cancha) ON DELETE CASCADE,
    FOREIGN KEY (f_temporada) REFERENCES Temporadas(f_inicio) ON DELETE CASCADE)

Done.


[]

In [130]:
%sql SELECT * FROM Entrenan

Done.


f_inicio,f_fin,entrenador,equipo,ciudad,comunidad,cancha,f_temporada


In [131]:
%%sql
INSERT INTO Entrenan(f_inicio, f_fin, entrenador, equipo, ciudad, comunidad, cancha, f_temporada)
VALUES ('2007-01-1 00:00:00','2007-12-31 00:00:00','66666666F','atletico','lugo', 'galicia', 'cancha1', '2007-01-1 00:00:00');
INSERT INTO Entrenan(f_inicio, f_fin, entrenador, equipo, ciudad, comunidad, cancha, f_temporada)
VALUES ('2008-01-1 00:00:00','2008-12-31 00:00:00','66666666F', 'real', 'alcobendas', 'madrid','cancha2','2008-01-1 00:00:00');

INSERT INTO Entrenan(f_inicio, f_fin, entrenador, equipo, ciudad, comunidad, cancha, f_temporada)
VALUES ('2007-01-1 00:00:00','2007-04-25 00:00:00','77777777G', 'real', 'alcobendas', 'madrid','cancha2','2007-01-1 00:00:00');
INSERT INTO Entrenan(f_inicio, f_fin, entrenador, equipo, ciudad, comunidad, cancha, f_temporada)
VALUES ('2008-01-1 00:00:00','2008-12-31 00:00:00','77777777G', 'deportivo', 'pontevedra', 'galicia','cancha1','2008-01-1 00:00:00');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [133]:
%sql SELECT * FROM Entrenan

Done.


f_inicio,f_fin,entrenador,equipo,ciudad,comunidad,cancha,f_temporada
2007-01-1 00:00:00,2007-12-31 00:00:00,66666666F,atletico,lugo,galicia,cancha1,2007-01-1 00:00:00
2008-01-1 00:00:00,2008-12-31 00:00:00,66666666F,real,alcobendas,madrid,cancha2,2008-01-1 00:00:00
2007-01-1 00:00:00,2007-04-25 00:00:00,77777777G,real,alcobendas,madrid,cancha2,2007-01-1 00:00:00
2008-01-1 00:00:00,2008-12-31 00:00:00,77777777G,deportivo,pontevedra,galicia,cancha1,2008-01-1 00:00:00


Añadimos ahora los puntos 13, 14 y 15. ![Diagrama E/R](Diagrama6.jpeg)

In [108]:
%sql DROP TABLE Partidos

Done.


[]

In [109]:
%%sql
CREATE TABLE Partidos(
    f_inicio DATETIME,
    f_fin DATETIME,
    
    equipo1 VARCHAR(30),
    ciudad1 VARCHAR(30),
    comunidad1 VARCHAR(30),
    
    equipo2 VARCHAR(30),
    ciudad2 VARCHAR(30),
    comunidad2 VARCHAR(30),
    
    cancha VARCHAR(30),
    
    f_temporada DATETIME,
    
    PRIMARY KEY (f_inicio,
                 equipo1, ciudad1, comunidad1, 
                 equipo2, ciudad2, comunidad2,
                 f_temporada),
    FOREIGN KEY (equipo1,ciudad1,comunidad1) REFERENCES Equipos(equipo,ciudad,comunidad) ON DELETE CASCADE,
    FOREIGN KEY (equipo2,ciudad2,comunidad2) REFERENCES Equipos(equipo,ciudad,comunidad) ON DELETE CASCADE,
    FOREIGN KEY (cancha) REFERENCES Canchas(cancha) ON DELETE CASCADE,
    FOREIGN KEY (f_temporada) REFERENCES Temporadas(f_inicio) ON DELETE CASCADE)

Done.


[]

In [113]:
%sql SELECT * FROM canchas

Done.


cancha
cancha1
cancha2
cancha3


In [114]:
%%sql 
INSERT INTO PARTIDOS (f_inicio, f_fin, equipo1, ciudad1, comunidad1, equipo2, ciudad2, comunidad2, cancha, f_temporada)
VALUES ('2007-01-10 16:00:00','2007-01-10 18:30:00','atletico','lugo', 'galicia', 'atletico','alcobendas', 'madrid','cancha1','2007-01-01 00:00:00' );
INSERT INTO PARTIDOS (f_inicio, f_fin, equipo1, ciudad1, comunidad1, equipo2, ciudad2, comunidad2, f_temporada)
VALUES ('2007-01-10 19:00:00', '2007-01-10 21:10:00','atletico','lugo', 'galicia', 'real','alcobendas', 'madrid','cancha2','2007-01-01 00:00:00' );
INSERT INTO PARTIDOS (f_inicio,f_fin, equipo1, ciudad1, comunidad1, equipo2, ciudad2, comunidad2, f_temporada)
VALUES ('2007-05-20 19:00:00', '2007-05-20 21:40:00','deportivo','pontevedra', 'galicia', 'real','alcobendas','madrid', 'cancha2','2007-01-01 00:00:00' );
INSERT INTO PARTIDOS (f_inicio, f_fin,equipo1, ciudad1, comunidad1, equipo2, ciudad2, comunidad2, f_temporada)
VALUES ('2008-05-23 15:00:00', '2008-05-23 17:20:00','deportivo','pontevedra', 'galicia', 'real','alcobendas', 'madrid', 'cancha3','2008-01-01 00:00:00' );


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "INSERT INTO PARTIDOS (f_inicio, f_fin, equipo1, ciudad1, comunidad1, equipo2, ciudad2, comunidad2, cancha, f_temporada)\nVALUES ('2007-01-10 16:00:00','2007-01-10 18:30:00','atletico','lugo', 'galicia', 'atletico','alcobendas', 'madrid','cancha1','2007-01-01 00:00:00' );"]

In [105]:
%sql SELECT * FROM Partidos

Done.


f_inicio,f_fin,equipo1,ciudad1,comunidad1,equipo2,ciudad2,comunidad2,cancha,f_temporada


Añadimos ahora el punto 16 y 19 (misma información). ![Diagrama E/R](Diagrama7.jpeg)

In [45]:
%%sql
CREATE TABLE Plantilla(
    jugador VARCHAR(30),
    equipo VARCHAR(30),
    ciudad VARCHAR(30),
    comunidad VARCHAR(30),    
    f_temporada DATETIME,
    PRIMARY KEY (jugador, equipo, ciudad, comunidad, f_temporada),
    FOREIGN KEY (jugador) REFERENCES Personas(nif) ON DELETE CASCADE,
    FOREIGN KEY (equipo) REFERENCES Equipos(equipo) ON DELETE CASCADE,
    FOREIGN KEY (ciudad) REFERENCES Ciudades(ciudad) ON DELETE CASCADE,
    FOREIGN KEY (comunidad) REFERENCES Comunidades(comunidad) ON DELETE CASCADE,
    FOREIGN KEY (f_temporada) REFERENCES Temporadas(f_inicio) ON DELETE CASCADE)

Done.


[]

In [46]:
%sql SELECT * FROM Plantilla

Done.


jugador,equipo,ciudad,comunidad,f_temporada


# Querys  - tercera entrega

# 4- Canchas en las que ha jugado un jugador a lo largo de una temporada.

In [145]:
%sql SELECT * FROM Plantilla

Done.


jugador,equipo,ciudad,comunidad,f_temporada


In [147]:
%sql SELECT * FROM Partidos

Done.


f_inicio,f_fin,equipo1,ciudad1,comunidad1,equipo2,ciudad2,comunidad2,cancha,f_temporada


In [146]:
%%sql
SELECT Partidos.cancha
    FROM Plantilla
    JOIN Partidos
    ON Plantilla.equipo = Partidos.equipo1 AND
        Plantilla.ciudad = Partidos.ciudad1 AND
        Plantilla.comunidad = Partidos.comunidad1
    OR Plantilla.equipo = Partidos.equipo2 AND
        Plantilla.ciudad = Partidos.ciudad2 AND
        Plantilla.comunidad = Partidos.comunidad2
    WHERE Jugador.jugador='11111111A' AND Entrenan.f_temporada='2007-01-1 00:00:00';

(sqlite3.OperationalError) near "ON": syntax error [SQL: "SELECT Partidos.cancha\n    FROM Plantilla\n    INNER Partidos\n    ON Plantilla.equipo = Partidos.equipo1 AND\n        Plantilla.ciudad = Partidos.ciudad1 AND\n        Plantilla.comunidad = Partidos.comunidad1\n    OR Plantilla.equipo = Partidos.equipo2 AND\n        Plantilla.ciudad = Partidos.ciudad2 AND\n        Plantilla.comunidad = Partidos.comunidad2\n    WHERE Jugador.jugador='11111111A' AND Entrenan.f_temporada='2007-01-1 00:00:00';"]


# 6 Ciudades en las que ha jugado un jugador a lo largo de una temporada.

In [ ]:
%%sql
CREATE VIEW Canchas_jugador_1_temp1 AS
    SELECT Partidos.cancha
    FROM Plantilla
    JOIN Partidos
    ON Plantilla.equipo = Partidos.equipo1 AND
        Plantilla.ciudad = Partidos.ciudad1 AND
        Plantilla.comunidad = Partidos.comunidad1
    OR Plantilla.equipo = Partidos.equipo2 AND
        Plantilla.ciudad = Partidos.ciudad2 AND
        Plantilla.comunidad = Partidos.comunidad2
    WHERE Jugador.jugador='11111111A' AND Entrenan.f_temporada='2007-01-1 00:00:00')

In [ ]:
%%sql
SELECT Direcciones_de_canchas.ciduad
    FROM Direcciones_de_canchas
    JOIN Canchas_jugador_1_temp1
    ON Direcciones_de_canchas.cancha = Canchas_jugador_1_temp1.cancha
    WHERE Direcciones_de_canchas.cancha = Canchas_jugador_1_temp1.cancha